#### Antibiotic Resistant Bacteria Multiclass-Classification and Drug Discovery
#### Corey J Sinnott
# Data Collection

## Executive Summary

This report was commissioned to determine a robust, fast, and reproducible means of searching for, and developing, new antibiotics, in an effort to combat antibiotic resistance. After in-depth analysis, conclusions and recommendations will be presented.
   
Data was obtained from the following source:
- Comprehensive Antibiotic Resistance Database via CARD CLI interface: 
 - https://card.mcmaster.ca
- ChEMBL via Python client library: 
 - https://www.ebi.ac.uk/chembl/ 

**Full Executive Summary, Conclusion, Recommendations, Data Dictionary and Sources can be found in README.**

## Contents:
- [Data Import & Cleaning](#Data-Import-&-Cleaning)

#### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from card_live_dashboard.service.CardLiveDataLoader import CardLiveDataLoader

# Data Import & Cleaning

Card Live accessed via terminal  
```python -m pip install card-live-dashboard```  
```virtualenv card-live-venv```  
```source card-live-venv```

In [ ]:
data_dir = Path('card-live-home/data/card_live')
loader = CardLiveDataLoader(data_dir)
data = loader.read_data()

#### Loading in all available dataframes

In [20]:
data.main_df.shape

(3723, 9)

In [28]:
main_df = data.main_df

In [29]:
rgi_df = data.rgi_df #RGI = resistance gene identifier

In [30]:
rgi_kmer_df = data.rgi_kmer_df

In [31]:
lmat_df = data.lmat_df

In [32]:
mlst_df = data.mlst_df

#### Inspecting Dataframes
- Isolating useful columns for merging

In [34]:
main_df.sample(2)

,timestamp,geo_area_code,geo_area_name,mlst_version,lmat_version,rgi_version,card_data_version,variants_data_version,analysis_valid
filename,,,,,,,,,
02c0fcc2c3f800200437218836265cf1,2020-08-10 08:46:47.068073,30,Eastern Asia,2.19.0,1.2.6,5.1.0,3.0.7,3.0.6,all
d04401fbb5712c1b194090d8715c62b6,2020-08-10 06:43:43.866792,30,Eastern Asia,2.19.0,1.2.6,5.1.0,3.0.7,3.0.6,all


In [35]:
rgi_df.sample(2)

,geo_area_name,mlst_version,lmat_version,rgi_version,card_data_version,variants_data_version,analysis_valid,rgi_main.ORF_ID,rgi_main.Cut_Off,rgi_main.Pass_Bitscore,...,rgi_main.Other_SNPs,rgi_main.Drug Class,rgi_main.Resistance Mechanism,rgi_main.AMR Gene Family,rgi_main.CARD_Protein_Sequence,rgi_main.Percentage Length of Reference Sequence,rgi_main.ID,rgi_main.Model_ID,rgi_main.Nudged,rgi_main.Note
filename,,,,,,,,,,,,,,,,,,,,,
cb4573cc4f661bf873a55c2afeb55521,Northern Europe,2.19.0,1.2.6,5.1.0,3.0.7,3.0.6,all,61.0,Strict,1400,...,None,fluoroquinolone antibiotic,antibiotic target alteration,fluoroquinolone resistant parC,MSDMAERLALHEFTENAYLNYSMYVIMDRALPFIGDGLKPVQRRIV...,100.00,gnl|BL_ORD_ID|2056|hsp_num:0,1450,,
c95696234d903e2d39c6fe23f6100d2a,Eastern Asia,2.19.0,1.2.6,5.1.1,3.1.0,3.0.7,all,53.0,Strict,375,...,None,macrolide antibiotic; fluoroquinolone antibiot...,antibiotic efflux,resistance-nodulation-cell division (RND) anti...,MRRTKEDSEKTRTAILLAAEELFLEKGVSHTSLEQIARAAGVTRGA...,100.00,gnl|BL_ORD_ID|2398|hsp_num:0,1213,,


In [48]:
rgi_use = rgi_df[['rgi_main.Drug Class', 'rgi_main.Resistance Mechanism', 
                 'rgi_main.CARD_Protein_Sequence', 'rgi_main.Percentage Length of Reference Sequence'
                ]]

In [50]:
rgi_kmer_df.sample(5)

,geo_area_name,mlst_version,lmat_version,rgi_version,card_data_version,variants_data_version,analysis_valid,rgi_kmer.CARD*kmer Prediction,rgi_kmer.Taxonomic kmers,rgi_kmer.Genomic kmers
filename,,,,,,,,,,
0e26c0ef73e340bcfcddd0e6b3d84773,Antarctica,2.19.0,1.2.6,5.1.0,3.0.7,3.0.6,all,Unknown taxonomy (chromosome or plasmid),,chr + plasmid: 832; plasmid: 0; chr: 293;
9ea94e1fea3af030bec282f461f0e29a,Southern Asia,2.19.0,1.2.6,5.1.0,3.0.7,3.0.6,all,Acinetobacter baumannii (chromosome or plasmid),Acinetobacter baumannii: 594;,chr + plasmid: 165; plasmid: 0; chr: 368;
77c39e525367a88bac0a9a064a0ca578,Northern Africa,2.19.0,1.2.6,5.1.1,3.1.0,3.0.7,all,Unknown taxonomy (plasmid),,chr + plasmid: 0; plasmid: 1062; chr: 0;
30043f39513056ec5cb91577f3a66dca,Western Asia,2.19.0,1.2.6,5.1.1,3.1.0,3.0.7,all,Unknown taxonomy (chromosome or plasmid),,chr + plasmid: 614; plasmid: 0; chr: 358;
f3f3f9ae155893cab7d190b4ab25f6a2,Eastern Asia,2.19.0,1.2.6,5.1.1,3.1.0,3.0.7,all,Pseudomonas aeruginosa (chromosome),Pseudomonas aeruginosa: 61; Pseudomonas: 228;,chr + plasmid: 0; plasmid: 0; chr: 729;


In [53]:
# come back to rgi_kmer_use = rgi_kmer_df['']

chr + plasmid: 0; plasmid: 0; chr: 1098;       1180
chr + plasmid: 0; plasmid: 0; chr: 270;         867
chr + plasmid: 0; plasmid: 0; chr: 1134;        853
chr + plasmid: 0; plasmid: 0; chr: 729;         842
chr + plasmid: 0; plasmid: 0; chr: 573;         824
                                               ... 
chr + plasmid: 676; plasmid: 3; chr: 0;           1
chr + plasmid: 5; plasmid: 0; chr: 68;            1
chr + plasmid: 0; plasmid: 0; chr: 424;           1
chr + plasmid: 1670; plasmid: 0; chr: 1393;       1
chr + plasmid: 22; plasmid: 0; chr: 179;          1
Name: rgi_kmer.Genomic kmers, Length: 8306, dtype: int64

In [37]:
lmat_df.sample(2)

,geo_area_name,mlst_version,lmat_version,rgi_version,card_data_version,variants_data_version,analysis_valid,lmat.score,lmat.count,lmat.ncbi_taxon_id,lmat.rank,lmat.taxonomy_label
filename,,,,,,,,,,,,
ab76421f14e816fc9079cb836fa7f5ed,Northern America,2.19.0,1.2.6,5.1.1,3.1.0,3.0.7,all,744.945,394,1381111,no rank,Corynebacterium pseudodiphtheriticum 090104
cd7dca366976d1af46a6e6e5e4eafccc,Southern Asia,2.19.0,1.2.6,5.1.0,3.0.7,3.0.6,all,4.19534,4,525243,no rank,Acinetobacter baumannii 6013150


In [110]:
lmat_use = lmat_df[['lmat.score', 'lmat.count', 'lmat.taxonomy_label']]

In [38]:
mlst_df.sample(2)

,geo_area_name,mlst_version,lmat_version,rgi_version,card_data_version,variants_data_version,analysis_valid,mlst.scheme,mlst.sequence_type,mlst.alleles_1,mlst.alleles_2,mlst.alleles_3,mlst.alleles_4,mlst.alleles_5,mlst.alleles_6,mlst.alleles_7,mlst.alleles_8,mlst.alleles_9,mlst.alleles_10
filename,,,,,,,,,,,,,,,,,,,
742acc1c33e107e10b0980627414b175,Northern Europe,2.19.0,1.2.6,5.1.1,3.1.0,3.0.6,rgi_main and mlst and lmat,suberis,None,arcC(3),ddl(4),gki(3),recP(2),tdk(5),tpi(2),yqiL(3),NaN,NaN,NaN
305a53c3ef5ce35950684abbf631f9ab,N/A,2.19.0,1.2.6,5.1.1,3.1.0,3.0.7,mlst and lmat,None,None,,,,,,,,NaN,NaN,NaN


In [56]:
mlst_use = mlst_df[['mlst.alleles_1', 'mlst.alleles_2', 'mlst.alleles_3', 
                   'mlst.alleles_4', 'mlst.alleles_5', 'mlst.alleles_6', 
                   'mlst.alleles_7', 'mlst.alleles_8', 'mlst.alleles_9', 
                   'mlst.alleles_10']]

In [73]:
rgi_use.shape

(91771, 4)

In [65]:
lmat_use.shape

(146567, 2)

#### Combining Dataframes

In [71]:
comb1 = rgi_use.join(lmat_use, how = 'outer')

In [72]:
comb1.shape

(4374969, 6)

In [74]:
comb1.head(2)

,rgi_main.Drug Class,rgi_main.Resistance Mechanism,rgi_main.CARD_Protein_Sequence,rgi_main.Percentage Length of Reference Sequence,lmat.score,lmat.count
filename,,,,,,
000e7e4842e3880db69532c0578e1960,cephalosporin; cephamycin; penam,antibiotic target alteration,MVKFNSSRKSGKSKKTIRKLTAPETVKQNKPQKVFEKCFMRGRYML...,98.52,10239.6,6686
000e7e4842e3880db69532c0578e1960,cephalosporin; cephamycin; penam,antibiotic target alteration,MVKFNSSRKSGKSKKTIRKLTAPETVKQNKPQKVFEKCFMRGRYML...,98.52,656.063,642


In [78]:
comb_trim = comb1.index.drop_duplicates()

In [79]:
comb_trim.shape

(3723,)

In [86]:
comb1.shape

(4374969, 6)

In [87]:
df = comb1.join(mlst_use, how = 'outer')

In [88]:
df.shape

(4374969, 16)

Trimming to only have one filename used 1x.

In [89]:
df = df[~df.index.duplicated()]

In [90]:
df.shape

(3723, 16)

In [91]:
df.sample(5)

,rgi_main.Drug Class,rgi_main.Resistance Mechanism,rgi_main.CARD_Protein_Sequence,rgi_main.Percentage Length of Reference Sequence,lmat.score,lmat.count,mlst.alleles_1,mlst.alleles_2,mlst.alleles_3,mlst.alleles_4,mlst.alleles_5,mlst.alleles_6,mlst.alleles_7,mlst.alleles_8,mlst.alleles_9,mlst.alleles_10
filename,,,,,,,,,,,,,,,,
d143a23860b0499de423400564cea425,cephalosporin,antibiotic inactivation,MRFKKISCLLLSPLFIFSTSIYAGNTPKDQEIKKLVDQNFKPLLEK...,100.00,17644.3,11856,Pas_cpn60(1),Pas_fusA(3),Pas_gltA(2),Pas_pyrG(1),Pas_recA(4),Pas_rplB(~4),Pas_rpoB(92),NaN,NaN,NaN
27003b51bd45251c4715c67966eb1c02,macrolide antibiotic; penam,antibiotic efflux,MAFYAFKAMRAAALAAAVALVLSSCGKGGDAAQGGQPAGREAPAPV...,100.00,6023.17,4269,abcZ(59),adk(39),aroE(67),fumC(78),gdh(148),pdhC(153),pgm(65),NaN,NaN,NaN
4608abe0a0f27769e34f4c456dbd9e53,fluoroquinolone antibiotic; tetracycline antib...,antibiotic efflux,MNISKFFIDRPIFAGVLSVLILLAGLLSVFQLPISEYPEVVPPSVV...,100.66,25058,19898,atpD(9),gltB(50),gyrB(84),recA(141),lepA(37),phaC(96),trpB(7),NaN,NaN,NaN
8cc96c0050719be943921bf2215ae10e,fluoroquinolone antibiotic; tetracycline antib...,antibiotic efflux,MFKKIFPLALVSSLRFLGLFIVLPVISLYADSFHSSSPLLVGLAVG...,100.00,29.3355,51,atpA(~1346),efp(~579),mutY(~267),ppa(~605),trpC(~2728),ureI(~2974),yphC(~375),NaN,NaN,NaN
d04401fbb5712c1b194090d8715c62b6,fluoroquinolone antibiotic; tetracycline antib...,antibiotic efflux,MNISKFFIDRPIFAGVLSVLILLAGLLSVFQLPISEYPEVVPPSVV...,100.57,12982.2,9444,,,,,,,,NaN,NaN,NaN


In [92]:
df['rgi_main.Resistance Mechanism'].value_counts()

antibiotic efflux                                              1777
antibiotic inactivation                                         636
antibiotic target alteration                                    283
antibiotic target replacement                                    91
antibiotic target protection                                     83
antibiotic target alteration; antibiotic efflux                  61
antibiotic efflux; reduced permeability to antibiotic            49
reduced permeability to antibiotic                               30
antibiotic target alteration; antibiotic target replacement       4
Name: rgi_main.Resistance Mechanism, dtype: int64

In [94]:
df[df['rgi_main.Resistance Mechanism'] == 'antibiotic target alteration; antibiotic efflux'] = df[df['rgi_main.Resistance Mechanism'] == 'antibiotic target alteration']

In [95]:
df['rgi_main.Resistance Mechanism'].value_counts()

antibiotic efflux                                              1777
antibiotic inactivation                                         636
antibiotic target alteration                                    283
antibiotic target replacement                                    91
antibiotic target protection                                     83
antibiotic efflux; reduced permeability to antibiotic            49
reduced permeability to antibiotic                               30
antibiotic target alteration; antibiotic target replacement       4
Name: rgi_main.Resistance Mechanism, dtype: int64

In [104]:
df['rgi_main.Resistance Mechanism'].replace(to_replace = 'antibiotic target alteration; antibiotic target replacement',
                                    value = 'antibiotic target alteration', inplace=True)

In [105]:
df['rgi_main.Resistance Mechanism'].value_counts()

antibiotic efflux                     1826
antibiotic inactivation                636
antibiotic target alteration           287
antibiotic target replacement           91
antibiotic target protection            83
reduced permeability to antibiotic      30
Name: rgi_main.Resistance Mechanism, dtype: int64

Exporting

In [109]:
#df.to_csv('trimmed_df_for_classification_v1.csv')

#### Bonus: Bulk collection for reference

In [113]:
# https://ecoliwiki.org/tools/chemgen/
# columns should be 'Gene' 'Antibiotic 1' 'Antibiotic 2' ... 'Antibiotic 234589612340596'
pd.read_csv('https://ecoliwiki.org/tools/chemgen/files/strainCC.txt')

Gene\tECK1963-HCHA\tECK1488-PQQL\tECK0813-YBIW\tECK2456-EUTP\tECK3716-BGLG\tECK1235-ADHE\tECK3926-CYTR\tECK2737-NLPD\tECK1700-PPS\tECK2623-YFJK\tECK1330-FNR\tECK3119-AGAR\tECK2272-NUOL\tECK2639-YFJX\tECK0434-HUPB\tECK3924-HSLV\tECK2281-NUOB\tECK2727-YGBA\tECK3512-YHJJ\tECK3225-MDH\tECK1562-YDFX\tECK0664-NAGC\tECK4170-HFLK\tECK3942-PFLD\tECK1602-YDGC\tECK3899-RHAR\tECK2175-YEJG\tECK4147-FRDD\tECK1665-YDHT\tECK1319-TYRR\tECK2236-YFAD\tECK2269-YFBP\tECK1115-PHOQ\tECK1809-YOAH\tECK0114-ACEF\tECK2350-YFDM'\tECK1884-CHEB\tECK1847-YEBE\tECK0710-SDHC\tECK2393-YFED\tECK2212-ATOS\tECK1905-YECH\tECK2780-BARA\tECK2002-YEEA\tECK1455-PPTA\tECK3660-UHPA\tECK2105-YEHE\tECK2148-CIRA\tECK4106-BASR\tECK2116-YEHS\tECK0761-YBHC\tECK0287-YKGJ\tECK1185-TREA\tECK0992-YCCE\tECK0355-YAIO\tECK0862-POXB\tECK2056-WZA\tECK0386-YAIE\tECK2930-TKTA\tECK1101-YCFT\tECK0405-TSX\tECK3062-AER\tECK1252-YCIF\tECK0757-YBHD\tECK0636-YBEL\tECK1324-YCJY\tECK1590-YNFL\tECK0690-YBFB\tECK0563-CUSR\tECK1357-YDAU\tECK0107-PPDD\tECK0787-YBIA\tECK1404-YNBD\tECK0488-TESA\tECK0849-YBJO\tECK1113-PEPT\tECK1021-YCDY\tECK1764-SPPA\tECK2457-EUTS\tECK1264-BTUR\tECK2322-MEPA\tECK1271-ACNA\tECK0830-DEOR\tECK2748-IAP\tECK2624-YFJL\tECK2800-FUCR\tECK3117-SOHA\tECK2273-NUOK\tECK2643-YPJF\tECK2912-ARGP\tECK3891-FRVX\tECK2282-NUOA\tECK2744-YGBE\tECK3483-PRLC\tECK3367-YHFW\tECK1565-YDFA\tECK4171-HFLC\tECK1621-CNU\tECK2218-YFAP\tECK4148-FRDC\tECK1668-YDHW\tECK1613-UIDR\tECK2237-YPAA\tECK0612-DPIB\tECK2285-YFBR\tECK1116-PHOP\tECK1818-YOBD\tECK0117-ACNB\tECK2352-YFDO'\tECK1889-CHEA\tECK1849-YEBG\tECK0711-SDHD\tECK2414-YFEK\tECK2213-ATOC\tECK1914-YECF\tECK3016-QSEB\tECK2007-YEEE\tECK1466-YDDL'\tECK3861-GLNG\tECK2109-YEHI\tECK1547-YDFP\tECK4117-DCUR\tECK1014-PGAA\tECK2043-CPSB\tECK0994-YCCJ\tECK1088-FHUE\tECK0360-YAIP\tECK1377-LDHA\tECK0794-FIU\tECK0398-ACPH\tECK2331\tECK3132-YRAJ\tECK0508-YLBA\tECK2102-YEHB\tECK1166-YCGK\tECK2854\tECK0559-YBCH\tECK1886-TAP\tECK1253-YCIG\tECK0638-YBER\tECK0255-YKFA\tECK1328-YNAJ\tECK1340-DBPA\tECK0692-YBFC\tECK3391-ENVZ\tECK0148-MRCB\tECK0791-YBIJ\tECK2509-YFGM\tECK1407-YDCF\tECK0867-YBJD\tECK2646-YPJC'\tECK1430-YNCJ\tECK2205-ADA\tECK1828-HTPX\tECK0893-PFLA\tECK2461-YPFG\tECK4127-CADC\tECK2381-YPDF\tECK1409-GAPC'\tECK2584-YFIM\tECK2360-DSDC\tECK2817-PTRA\tECK1853-ZWF\tECK2625-YFJM\tECK2835-GALR\tECK3146-YHBU\tECK2274-NUOJ\tECK2650\tECK4257-IDNR\tECK3839-PEPQ\tECK2384-GLK\tECK4111-MELR\tECK3453-ZNTA\tECK3373-RPE\tECK1567-RZPQ'\tECK0021-NHAR\tECK4253-PEPA\tECK3947-PPC\tECK1622-YDGK\tECK4055-SOXR\tECK2219-YFAQ\tECK4149-FRDB\tECK1682-YDIH\tECK4315-UXUR\tECK2238-YFAU\tECK0613-DPIA\tECK0081-FRUR\tECK2293-YFCD\tECK1215-NARL\tECK1823-YEBO\tECK0330-PRPC\tECK2354-YFDQ\tECK1913-UVRY\tECK0712-SDHA\tECK2427-YFEY\tECK2365-EVGA\tECK1927-YEDD\tECK3017-QSEC\tECK2067-YEGL\tECK1467-YDDG\tECK3862-GLNL\tECK2110-YEHK\tECK2809-MLTA\tECK0252-YKFB\tECK4118-DCUS\tECK3573-YIAT\tECK0289-YAGW\tECK3745-RBSB\tECK0361-YAIS\tECK2126-DLD\tECK1048-YCEB\tECK0564-CUSC\tECK0404-YAJD\tECK2339-YFDF\tECK1133-YMFL\tECK2053-WCAA\tECK1182-YCGR\tECK0565-CUSF\tECK1797-YEAT\tECK0060-HEPA\tECK3416-GLGC\tECK1363-YNAK\tECK0160-DEGP\tECK0792-YBII\tECK1411-YDCA\tECK0625-DACA\tECK0910-YCBJ\tECK1432-YDCQ\tECK1171-UMUD\tECK1041-YCEI\tECK4109-ADIY\tECK0894-PFLB\tECK2468-YPFN\tECK1982-CBL\tECK2487-HYFR\tECK3065-EBGR\tECK2868-YGEY\tECK1855-PYKA\tECK2626-RNLA\tECK2144-GALS\tECK3171-DACB\tECK3765-ILVY\tECK2458-MAEB\tECK2750-YGBT\tECK3930-METJ\tECK3431-GGT\tECK3390-PCK\tECK1570-YDFD\tECK0112-PDHR\tECK4006-ACEB\tECK1635-MLIC\tECK4054-SOXS\tECK4150-FRDA\tECK1703-YDIE\tECK2399-XAPR\tECK1710-IHFA\tECK2299-YFCI\tECK1216-NARX\tECK0423-CYOD\tECK2355-YFDR\tECK1964-YEDV\tECK1869-YECE\tECK0713-SDHB\tECK2428-YFEZ\tECK2366-EVGS\tECK1928-YEDE\tECK3191-RPON\tECK3904-CPXA\tECK2696-MLTB\tECK4390-CREB\tECK1804-YEAY\tECK0291-MATC\tECK1237-OPPA\tECK2473-BAMC\tECK0370-YAIV\tECK3595-LLDD\tECK1052-YCEH\tECK4028-LAMB\tECK1134-YMFM\tECK1193-YCHH\tECK0123-GCD\tECK0953-HELD\tECK1267-YCIN\tECK2150-YEIE\tECK0641-YBEU\tECK1337-YDAL\tECK0147-HRPB\tECK24

In [20]:
columns = ['prevalence_sequence_id','model_id','aro_term','aro_accession','detection_model','species_name','ncbi_accession','data_type','rgi_criteria','percent_identity','bitscore','amr_gene_family','resistance_mechanism','drug_class'];
test2 = pd.read_csv('./antibiotics/index-for-model-sequences.txt', sep='\\t', header=None, names = columns)

<ipython-input-20-e4215bcf4770>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test2 = pd.read_csv('./antibiotics/index-for-model-sequences.txt', sep='\\t', header=None, names = columns)


In [21]:
test2.head(5)

,prevalence_sequence_id,model_id,aro_term,aro_accession,detection_model,species_name,ncbi_accession,data_type,rgi_criteria,percent_identity,bitscore,amr_gene_family,resistance_mechanism,drug_class
0,prevalence_sequence_id,model_id,aro_term,aro_accession,detection_model,species_name,ncbi_accession,data_type,rgi_criteria,percent_identity,bitscore,amr_gene_family,resistance_mechanism,drug_class
1,1,3280,Acinetobacter baumannii AmvA,ARO:3004577,protein homolog model,Acinetobacter nosocomialis,NZ_AWOW01,ncbi_contig,Strict,95.73,874.8,major facilitator superfamily (MFS) antibiotic...,antibiotic efflux,acridine dye; macrolide antibiotic
2,1,3280,Acinetobacter baumannii AmvA,ARO:3004577,protein homolog model,Acinetobacter nosocomialis,NZ_CP040105.1,ncbi_chromosome,Strict,95.73,874.8,major facilitator superfamily (MFS) antibiotic...,antibiotic efflux,acridine dye; macrolide antibiotic
3,1,3280,Acinetobacter baumannii AmvA,ARO:3004577,protein homolog model,Acinetobacter nosocomialis,NZ_NGIH01,ncbi_contig,Strict,95.73,874.8,major facilitator superfamily (MFS) antibiotic...,antibiotic efflux,acridine dye; macrolide antibiotic
4,2,593,abeS,ARO:3000768,protein homolog model,Acinetobacter nosocomialis,NZ_APCE01,ncbi_contig,Perfect,100,203,small multidrug resistance (SMR) antibiotic ef...,antibiotic efflux,aminocoumarin antibiotic; macrolide antibiotic


In [22]:
test2[['aro_accession']].describe()

,aro_accession
count,2955819
unique,1907
top,ARO:3003369
freq,59789


In [44]:
test.shape

(2955818, 2)